In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from matplotlib import pyplot as plt
from sklearn import preprocessing 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv


In [2]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='gabe-lmps'
data_key = 'mean_hourly_solar2.csv'
data_location = 's3://{0}/{1}'.format(bucket, data_key)
mea_hr_s = pd.read_csv(data_location)

In [3]:
mea_hr_s=mea_hr_s.drop(['pnode_id', 'row_is_current', 'version_nbr', 'is_renewable'], axis=1)

In [4]:
mea_hr_s=mea_hr_s.set_index('datetime')
mea_hr_s.head()

,system_energy_price_rt,total_lmp_rt,congestion_price_rt,marginal_loss_price_rt,solar_generation_mw,mw,fuel_percentage_of_total
datetime,,,,,,,
2020-07-01 00:00:00,15.65,15.672963,0.00,0.019676,-1.153,21.9,0.0
2020-07-01 01:00:00,15.24,15.276157,0.00,0.034815,-1.157,21.9,0.0
2020-07-01 02:00:00,14.43,14.498889,0.03,0.040324,-1.190,21.9,0.0
2020-07-01 03:00:00,13.66,13.704028,0.00,0.041343,-1.172,21.9,0.0
2020-07-01 04:00:00,13.44,13.482500,0.00,0.041343,-1.180,21.9,0.0


In [6]:
mea_hr_s.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2185 entries, 2020-07-01 00:00:00 to 2020-09-30 00:00:00
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   system_energy_price_rt    2185 non-null   float64
 1   total_lmp_rt              2185 non-null   float64
 2   congestion_price_rt       2185 non-null   float64
 3   marginal_loss_price_rt    2185 non-null   float64
 4   solar_generation_mw       2185 non-null   float64
 5   mw                        2185 non-null   float64
 6   fuel_percentage_of_total  2185 non-null   float64
dtypes: float64(7)
memory usage: 136.6+ KB


In [7]:
mea_hr_s.describe()

,system_energy_price_rt,total_lmp_rt,congestion_price_rt,marginal_loss_price_rt,solar_generation_mw,mw,fuel_percentage_of_total
count,2185.000000,2185.000000,2185.000000,2185.000000,2185.000000,2185.000000,2185.000000
mean,22.347618,24.347896,1.836495,0.164160,182.722933,463.241876,0.003950
std,13.251177,17.260813,9.163961,0.319137,233.958632,562.932264,0.005439
min,4.910000,-6.249630,-64.935093,-1.163102,-3.225000,0.000000,0.000000
25%,14.870000,15.684630,0.000000,0.008287,-1.163000,10.300000,0.000000
50%,18.710000,19.944815,0.210972,0.129398,25.772000,54.700000,0.000000
75%,24.640000,25.979583,2.501759,0.293333,360.795000,964.500000,0.010000
max,129.150000,236.143056,158.687176,2.445648,787.953000,1761.100000,0.020000


In [5]:

 
# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]
 
# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]
 
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error


#score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)
 
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores
 
# create a set of sarima configs to try
def sarima_configs(seasonal=[24]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models


In [ ]:



if __name__ == '__main__':
	# load dataset
	data = mea_hr_s["total_lmp_rt"].values
	print(data.shape)
	# data split
	n_test = 1043
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)


(2185,)
 > Model[[(0, 0, 0), (0, 0, 0, 24), 'n']] 26.972
 > Model[[(0, 0, 0), (0, 1, 0, 24), 'n']] 16.199
 > Model[[(0, 0, 0), (1, 0, 0, 24), 'n']] 15.487
 > Model[[(0, 0, 0), (1, 1, 0, 24), 'n']] 14.966
 > Model[[(0, 0, 1), (0, 0, 0, 24), 'n']] 19.385
 > Model[[(0, 0, 0), (0, 0, 1, 24), 'n']] 20.418
 > Model[[(0, 0, 1), (0, 1, 0, 24), 'n']] 15.525
 > Model[[(0, 0, 1), (0, 0, 1, 24), 'n']] 16.818
 > Model[[(0, 0, 0), (2, 0, 0, 24), 'n']] 14.648
 > Model[[(0, 0, 1), (1, 0, 0, 24), 'n']] 14.521
 > Model[[(0, 0, 0), (2, 1, 0, 24), 'n']] 14.119


In [8]:

data = mea_hr_s["total_lmp_rt"].values
print(data.shape)
# data split
n_test = 1043

(2185,)


In [8]:
'''
cfg4 = [(0,0,0), (1,1,0,24), 'n']
model4 = walk_forward_validation(data, n_test, cfg4)
print(model4)
''''''

"\ncfg4 = [(0,0,0), (1,1,0,24), 'n']\nmodel4 = walk_forward_validation(data, n_test, cfg4)\nprint(model4)\n"

In [9]:
'''
C
model5 = walk_forward_validation(data, n_test, cfg5)
print(model5)
'''

'\nC\nmodel5 = walk_forward_validation(data, n_test, cfg5)\nprint(model5)\n'

In [ ]:
### experimenting w/ a higher order parameter 
cfg4 = [(0,0,0), (2,1,1,24), 'n']
model4 = walk_forward_validation(data, n_test, cfg4)
print(model4)